# Solving a linear problem

Example of implementing a linear problem.
We'll use the biharmonic spline interpolation problem.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
def make_data(x):
    return (2*np.sin(x) - 2.5*np.sin(0.5*(x - 30)) + 3*np.cos(0.3*(x))
            + 0.5*np.sin(3*x))

In [ ]:
np.random.seed(5)
x = np.linspace(0, 20, 100)
y = make_data(x)
x_obs = np.random.choice(x, size=40)
y_obs = make_data(x_obs)

plt.figure()
plt.plot(x_obs, y_obs, 'x')
plt.plot(x, y, '-')

In [ ]:
from deeplook import LinearMisfit
from deeplook.regularization import Damping

In [ ]:
class BiharmonicSpline1D():
    
    def __init__(self, damping=None):
        self.damping = damping
    
    def jacobian(self, x):
        ndata = x.size
        jac = np.empty((ndata, self.nparams), dtype=np.float64)
        for i in range(ndata):
            jac[i, :] = np.abs(x[i] - self.x_forces)**3
        return jac
    
    def predict(self, x):
        return self.jacobian(x).dot(self.params_)
    
    def fit(self, x, y):
        self.x_forces = x
        self.nparams = x.size
        jacobian = self.jacobian(x)
        regul = []
        if self.damping is not None:
            regul.append(Damping(self.damping, self.nparams))
        self.misfit = LinearMisfit(data=y, jacobian=jacobian, normalize=True,
                                   regularization=regul)
        self.params_ = self.misfit.minimize()
        return self

In [ ]:
interp = BiharmonicSpline1D(damping=1e-3).fit(x_obs, y_obs)

plt.figure()
plt.plot(x_obs, y_obs, 'x')
plt.plot(x, interp.predict(x), '-r')
plt.plot(x, y, '--b')

In [ ]:
class BiharmonicSpline1DCV(BiharmonicSpline1D):
    def __init__(self, dampings):
        super().__init__()
        self.dampings = dampings

    def fit(self, x, y):
        size = len(self.dampings)
        self.scores = np.empty(size, dtype=np.float)
        folds = KFold()
        for i, damping in enumerate(self.dampings):
            score = 0
            spline = BiharmonicSpline1D(damping=damping)
            for train, test in folds.split(x):
                spline.fit(x[train], y[train])
                score += mean_squared_error(y[test], spline.predict(x[test])) 
            self.scores[i] = score/folds.n_splits
        self.damping = self.dampings[np.argmin(self.scores)]
        super().fit(x, y)
        return self
        

In [ ]:
dampings = np.array([10**(i) for i in range(-14, 0)])
spline_cv = BiharmonicSpline1DCV(dampings).fit(x_obs, y_obs)

In [ ]:
plt.figure()
plt.plot(dampings, spline_cv.scores)
plt.xscale('log')
plt.yscale('log')

In [ ]:
plt.figure()
plt.plot(x_obs, y_obs, 'x')
plt.plot(x, interp.predict(x), '-r')
plt.plot(x, spline_cv.predict(x), '--g')
plt.plot(x, y, '--b')